# CardioML

Elaborato di Programmazione di Applicazioni Data Intensive - 2023

Membri del Gruppo:
- Matteo Iorio
- Fabio Vincenzi
- Stefano Furi

Il dataset utilizzato riguarda una serie di misurazioni effettuate attraverso [cardiotocografia](https://it.wikipedia.org/wiki/Cardiotocografia), consultabile presso [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/cardiotocography#).

Lo scopo di questo elaborato verrà scritto qui molto presto :)

Decidiamo se mettere il `setup.py` con link qui direttamente alla repo github, così se i prof chiedono solo il file jupyter, possono tranquillamente usarlo senza clonare l'intera repo, mentre invece per i temerari che clonano l'intera repo, il dataset è già qui integrato.

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [62]:
raw_df = pd.read_csv("../data/ctg.csv", delimiter=';')

In [63]:
raw_df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,01/12/96,CTG0001.txt,240,357,120,120,0,0,0,...,0,0,0,0,0,0,1,0,9,2
1,Fmcs_1.txt,03/05/96,CTG0002.txt,5,632,132,132,4,0,4,...,0,0,0,1,0,0,0,0,6,1
2,Fmcs_1.txt,03/05/96,CTG0003.txt,177,779,133,133,2,0,5,...,0,0,0,1,0,0,0,0,6,1
3,Fmcs_1.txt,03/05/96,CTG0004.txt,411,1192,134,134,2,0,6,...,0,0,0,1,0,0,0,0,6,1
4,Fmcs_1.txt,03/05/96,CTG0005.txt,533,1147,132,132,4,0,5,...,0,0,0,0,0,0,0,0,2,1


SPIEGAZIONE RAPIDA DELLE VARIABILI, DIVISE PER CATEGORICHE E CONTINUE

Mappiamo i nomi di aclune colonne in nomi più esplicativi.

In [64]:
col_names = {
    "FileName": "file_name",
    "Date": "date",
    "SegFile": "seg_file",
    "b": "start_instant",
    "e": "end_instant",
    "LBE": "medical_baseline",
    "LB": "sisproto_baseline",
    "AC": "heart_acc",
    "FM": "foetal_move",
    "UC": "utherine_contract",
    "ASTV": "pct_short_variability",
    "MSTV": "mean_short_variability",
    "ALTV": "pct_long_variability",
    "MLTV": "mean_long_variability",
    "DL": "light_dec",
    "DS": "severe_dec",
    "DP": "prolongued_dec",
    "DR": "repetitive_dec",
    "A": "calm_sleep",
    "B": "rem_sleep",
    "C": "calm_vigilance",
    "D": "active_vigilance",
    "SH": "shift_pattern",
    "AD": "acc_pattern_stress", # negativo se pattern in decelerazione
    "DE": "dec_pattern_vagal",
    "LD": "largely_dec",
    "FS": "patho_flat_sinusoidal",
    "SUSP": "suspect"
}

In [65]:
raw_df.rename(columns=col_names, inplace=True)
raw_df.columns = raw_df.columns.str.lower()
raw_df.columns

Index(['file_name', 'date', 'seg_file', 'start_instant', 'end_instant',
       'medical_baseline', 'sisproto_baseline', 'heart_acc', 'foetal_move',
       'utherine_contract', 'pct_short_variability', 'mean_short_variability',
       'pct_long_variability', 'mean_long_variability', 'light_dec',
       'severe_dec', 'prolongued_dec', 'repetitive_dec', 'width', 'min', 'max',
       'nmax', 'nzeros', 'mode', 'mean', 'median', 'variance', 'tendency',
       'calm_sleep', 'rem_sleep', 'calm_vigilance', 'active_vigilance', 'e',
       'acc_pattern_stress', 'dec_pattern_vagal', 'largely_dec',
       'patho_flat_sinusoidal', 'suspect', 'class', 'nsp'],
      dtype='object')

Alcune variabili categoriche sono state mappate a `np.float64`, mentre altre sarebbe stato sufficiente un tipo `np.int32`. Possiamo quindi ricaricare il dataset, specificando il tipo di dato per ogni colonna, facendo anche riferimento all'utilizzo di memoria

In [66]:
raw_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   file_name               2126 non-null   object
 1   date                    2126 non-null   object
 2   seg_file                2126 non-null   object
 3   start_instant           2126 non-null   int64 
 4   end_instant             2126 non-null   int64 
 5   medical_baseline        2126 non-null   int64 
 6   sisproto_baseline       2126 non-null   int64 
 7   heart_acc               2126 non-null   int64 
 8   foetal_move             2126 non-null   int64 
 9   utherine_contract       2126 non-null   int64 
 10  pct_short_variability   2126 non-null   int64 
 11  mean_short_variability  2126 non-null   object
 12  pct_long_variability    2126 non-null   int64 
 13  mean_long_variability   2126 non-null   object
 14  light_dec               2126 non-null   int64 
 15  seve

In [94]:
bool_vars = [
    "A", "B", "C", "D", "SH",
    "AD", "DE", "LD", "FS", "SUSP"
]

cat_vars = ["CLASS", "NSP", "Tendency"]

num_vars = [
    "b", "e", "LBE", "LB", "AC",
    "FM", "UC", "ASTV",
    "MSTV", "ALTV", "MLTV", "DL",
    "DS", "DP", "DR", "Width", "Min",
    "Max", "Nmax", "Nzeros", "Mode",
    "Mean", "Median", "Variance"
]

custom_dtpes = { 
    **{ col: "bool" for col in bool_vars }, 
    **{ col: "category" for col in cat_vars}
}

In [95]:
raw_df = pd.read_csv("../data/ctg.csv", delimiter=';', dtype=custom_dtpes)

In [96]:
raw_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 40 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   FileName  2126 non-null   object  
 1   Date      2126 non-null   object  
 2   SegFile   2126 non-null   object  
 3   b         2126 non-null   int64   
 4   e         2126 non-null   int64   
 5   LBE       2126 non-null   int64   
 6   LB        2126 non-null   int64   
 7   AC        2126 non-null   int64   
 8   FM        2126 non-null   int64   
 9   UC        2126 non-null   int64   
 10  ASTV      2126 non-null   int64   
 11  MSTV      2126 non-null   object  
 12  ALTV      2126 non-null   int64   
 13  MLTV      2126 non-null   object  
 14  DL        2126 non-null   int64   
 15  DS        2126 non-null   int64   
 16  DP        2126 non-null   int64   
 17  DR        2126 non-null   int64   
 18  Width     2126 non-null   int64   
 19  Min       2126 non-null   int64   
 20  Max     

Abbiamo risparmiato 0.1 MB ! (amazzati)